# Overview

In [ ]:
-- Employee HR Analytics EDA
-- Comment out USE statement for now
-- USE perf_prod;

-- 1. Basic overview of the dataset
SELECT COUNT(*) AS total_employees FROM employees;

-- 2. Understanding the data structure
DESC employees;

-- 3. Sample of data (first 10 rows)
SELECT * FROM employees LIMIT 10;


The first 10 rows of the dataset were sampled to inspect the data for any anomalies.

The basic structure of the table was investigated using the describe keyword to figure out the datatypes for the various columns

In [ ]:
select * from employees;

# Data Quality Checks

## Check for missing values

In [ ]:
-- Check for missing values in key columns
SELECT 
    COUNT(*) AS total_records,
    SUM(CASE WHEN Employee_ID IS NULL THEN 1 ELSE 0 END) AS missing_employee_id,
    SUM(CASE WHEN Department IS NULL THEN 1 ELSE 0 END) AS missing_department,
    SUM(CASE WHEN Gender IS NULL THEN 1 ELSE 0 END) AS missing_gender,
    SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) AS missing_age,
    SUM(CASE WHEN Monthly_Salary IS NULL THEN 1 ELSE 0 END) AS missing_salary,
    SUM(CASE WHEN Performance_Score IS NULL THEN 1 ELSE 0 END) AS missing_performance,
    SUM(CASE WHEN Employee_Satisfaction_Score IS NULL THEN 1 ELSE 0 END) AS missing_satisfaction,
    SUM(CASE WHEN Resigned IS NULL THEN 1 ELSE 0 END) AS missing_resignation
FROM employees;

Insights: No missing values were found in the dataset.



## Check for duplicate records

In [ ]:
-- Duplicate Records Check
-- Purpose: Identify any duplicate employee records
SELECT 
    Employee_ID, 
    COUNT(*) AS record_count
FROM employees
GROUP BY Employee_ID
HAVING COUNT(*) > 1;

Insights: No duplicate values were found

## Check for outliers

In [ ]:
-- Value Range Check
-- Purpose: Identify values outside expected ranges
SELECT
    COUNT(*) AS total_records,
    SUM(CASE WHEN Age < 18 OR Age > 70 THEN 1 ELSE 0 END) AS invalid_age,
    SUM(CASE WHEN Monthly_Salary < 0 THEN 1 ELSE 0 END) AS negative_salary,
    SUM(CASE WHEN Performance_Score < 1 OR Performance_Score > 5 THEN 1 ELSE 0 END) AS invalid_performance,
    SUM(CASE WHEN Employee_Satisfaction_Score < 0 OR Employee_Satisfaction_Score > 5 THEN 1 ELSE 0 END) AS invalid_satisfaction,
    SUM(CASE WHEN Resigned NOT IN (0, 1) THEN 1 ELSE 0 END) AS invalid_resignation
FROM employees;

Insights: no outliers were found

# DATA STRUCTURE & SUMMARY STATISTICS

In [ ]:
-- 2.1: Table Structure
-- Purpose: Understand the data types and structure of the employees table
DESCRIBE employees;

## Basic Summary Statistics (Numerical Columns)

In [ ]:
--  Basic Summary Statistics
-- Purpose: Calculate descriptive statistics for numerical columns
SELECT
    'Age' AS column_name,
    COUNT(Age) AS count,
    MIN(Age) AS minimum,
    MAX(Age) AS maximum,
    ROUND(AVG(Age), 2) AS mean,
    ROUND(STDDEV(Age), 2) AS std_dev
FROM employees

UNION ALL

SELECT
    'Monthly_Salary' AS column_name,
    COUNT(Monthly_Salary) AS count,
    MIN(Monthly_Salary) AS minimum,
    MAX(Monthly_Salary) AS maximum,
    ROUND(AVG(Monthly_Salary), 2) AS mean,
    ROUND(STDDEV(Monthly_Salary), 2) AS std_dev
FROM employees

UNION ALL

SELECT
    'Performance_Score' AS column_name,
    COUNT(Performance_Score) AS count,
    MIN(Performance_Score) AS minimum,
    MAX(Performance_Score) AS maximum,
    ROUND(AVG(Performance_Score), 2) AS mean,
    ROUND(STDDEV(Performance_Score), 2) AS std_dev
FROM employees

UNION ALL

SELECT
    'Employee_Satisfaction_Score' AS column_name,
    COUNT(Employee_Satisfaction_Score) AS count,
    MIN(Employee_Satisfaction_Score) AS minimum,
    MAX(Employee_Satisfaction_Score) AS maximum,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS mean,
    ROUND(STDDEV(Employee_Satisfaction_Score), 2) AS std_dev
FROM employees;

# Basic Summary Statistics (categorical columns)

In [ ]:
-- Purpose: Analyze the distribution of categorical variables
-- Department Distribution
SELECT 
    Department, 
    COUNT(*) AS count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY Department
ORDER BY count DESC;

In [ ]:
-- Gender Distribution
SELECT 
    Gender, 
    COUNT(*) AS count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY Gender
ORDER BY count DESC;

In [ ]:
-- Education Level Distribution
SELECT 
    Education_Level, 
    COUNT(*) AS count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY Education_Level
ORDER BY count DESC;

In [ ]:
-- Resignation Status Distribution
SELECT 
    CASE WHEN Resigned = 1 THEN 'Resigned' ELSE 'Active' END AS status,
    COUNT(*) AS count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY Resigned
ORDER BY status;

# Distribution Analysis

In [ ]:
-- Age Distribution
-- Purpose: Analyze age distribution in 5-year brackets
SELECT 
    age_bracket,
    CONCAT(age_bracket, '-', age_bracket+4) AS age_range,
    COUNT(*) AS employee_count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM (
    SELECT 
        FLOOR(Age/5)*5 AS age_bracket
    FROM employees
) AS age_groups
GROUP BY age_bracket
ORDER BY age_bracket;

In [ ]:
-- Salary Distribution
-- Purpose: Analyze salary distribution in $1000 brackets
SELECT 
    salary_bracket,
    CONCAT('$', salary_bracket, '-$', salary_bracket+999) AS salary_range,
    COUNT(*) AS employee_count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM (
    SELECT 
        FLOOR(Monthly_Salary/1000)*1000 AS salary_bracket
    FROM employees
) AS salary_groups
GROUP BY salary_bracket
ORDER BY salary_bracket;

In [ ]:
-- Performance Score Distribution
-- Purpose: Analyze the distribution of performance scores
SELECT 
    Performance_Score,
    COUNT(*) AS employee_count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY Performance_Score
ORDER BY Performance_Score;

In [ ]:
-- Satisfaction Score Distribution
-- Purpose: Analyze the distribution of satisfaction scores in ranges
SELECT 
    CASE 
        WHEN Employee_Satisfaction_Score >= 4 THEN 'Very Satisfied (4-5)'
        WHEN Employee_Satisfaction_Score >= 3 THEN 'Satisfied (3-3.99)'
        WHEN Employee_Satisfaction_Score >= 2 THEN 'Neutral (2-2.99)'
        WHEN Employee_Satisfaction_Score >= 1 THEN 'Dissatisfied (1-1.99)'
        ELSE 'Very Dissatisfied (<1)'
    END AS satisfaction_level,
    COUNT(*) AS employee_count,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM employees) * 100, 2) AS percentage
FROM employees
GROUP BY satisfaction_level
ORDER BY MIN(Employee_Satisfaction_Score) DESC;

# Departmental Analysis

In [ ]:
-- Department Demographics
-- Purpose: Compare key metrics across departments
SELECT 
    Department,
    COUNT(*) AS employee_count,
    ROUND(AVG(Age), 1) AS avg_age,
    ROUND(AVG(Monthly_Salary), 2) AS avg_salary,
    ROUND(AVG(Performance_Score), 1) AS avg_performance,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS avg_satisfaction
FROM employees
GROUP BY Department
ORDER BY employee_count DESC;

In [ ]:
--Department Salary Analysis
-- Purpose: Detailed salary analysis by department
SELECT 
    Department,
    COUNT(*) AS employee_count,
    ROUND(AVG(Monthly_Salary), 2) AS avg_salary,
    ROUND(MIN(Monthly_Salary), 2) AS min_salary,
    ROUND(MAX(Monthly_Salary), 2) AS max_salary,
    ROUND(STDDEV(Monthly_Salary), 2) AS salary_deviation,
    ROUND((AVG(Monthly_Salary) / (SELECT AVG(Monthly_Salary) FROM employees) * 100 - 100), 1) AS percent_diff_from_avg
FROM employees
GROUP BY Department
ORDER BY avg_salary DESC;

In [ ]:
-- Department Retention Analysis
-- Purpose: Analyze satisfaction and attrition by department
SELECT 
    Department,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS avg_satisfaction,
    COUNT(*) AS total_employees,
    SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) AS resignation_count,
    ROUND(SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) AS resignation_rate,
    ROUND(AVG(CASE WHEN Resigned = 1 THEN Employee_Satisfaction_Score ELSE NULL END), 2) AS resigned_avg_satisfaction,
    ROUND(AVG(CASE WHEN Resigned = 0 THEN Employee_Satisfaction_Score ELSE NULL END), 2) AS active_avg_satisfaction,
    ROUND(AVG(Years_At_Company), 1) AS avg_tenure
FROM employees
GROUP BY Department
ORDER BY resignation_rate DESC;

# Education Analysis

In [ ]:
-- Education Impact Analysis
-- Purpose: Analyze how education affects key metrics
SELECT 
    Education_Level,
    COUNT(*) AS employee_count,
    ROUND(AVG(Performance_Score), 1) AS avg_performance,
    ROUND(AVG(Monthly_Salary), 2) AS avg_salary,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS avg_satisfaction,
    SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) AS resignation_count,
    ROUND(SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) AS resignation_rate
FROM employees
GROUP BY Education_Level
ORDER BY avg_performance DESC;

# Remote work analysis

In [ ]:
-- Remote Work Impact
-- Purpose: Analyze how remote work affects satisfaction and retention
SELECT 
    CASE
        WHEN Remote_Work_Frequency = 0 THEN 'Fully In-Office (0%)'
        WHEN Remote_Work_Frequency = 25 THEN 'Low Remote (25%)'
        WHEN Remote_Work_Frequency = 50 THEN 'Hybrid (50%)'
        WHEN Remote_Work_Frequency = 75 THEN 'Mostly Remote (75%)'
        WHEN Remote_Work_Frequency = 100 THEN 'Fully Remote (100%)'
        ELSE 'Other'
    END AS remote_work_category,
    COUNT(*) AS employee_count,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS avg_satisfaction,
    ROUND(AVG(Performance_Score), 1) AS avg_performance,
    SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) AS resignation_count,
    ROUND(SUM(CASE WHEN Resigned = 1 THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) AS resignation_rate
FROM employees
GROUP BY remote_work_category
ORDER BY MIN(Remote_Work_Frequency);


# Attrition analysis

In [ ]:
-- Department-wise attrition
SELECT
    Department,
    COUNT(*) AS total_employees,
    SUM(Resigned) AS resignations,
    ROUND(SUM(Resigned) / COUNT(*) * 100, 1) AS resignation_rate,
    ROUND(AVG(Employee_Satisfaction_Score), 2) AS avg_satisfaction
FROM
    employees
GROUP BY
    Department
ORDER BY
    resignation_rate DESC;